<a href="https://colab.research.google.com/github/artturner/SDS-CP008-superstore-customer-segmentation/blob/main/Art_SuperStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
from google.colab import files
uploaded = files.upload()

Saving Sample - Superstore.csv to Sample - Superstore.csv


In [5]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Sample - Superstore.csv']), encoding='latin1')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [6]:
pd.set_option('display.max_columns',50)

df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


List all Features and provide a definition

*   Row ID. Row number.
*   Order ID.
*   Order Date
*   Ship Date
*   Ship Mode: Class- First, Second...
*   Customer ID
*   Customer Segment
*   Country
*   City
*   State
*   Postal Code
*   Region
*   Product ID
*   Category
*   Sub-Category
*   Product Name
*   Sales - Total Price of products sold
*   Quantity - Number of items sold
*   Discount - Represents the rate applied at the order level?
*   Profit

In [7]:
print('The dataset has {} rows and {} columns.'.format(df.shape[0],df.shape[1]))

The dataset has 9994 rows and 21 columns.


In [8]:
df.shape

(9994, 21)

In [9]:
df.isna().sum()

,0
Row ID,0
Order ID,0
Order Date,0
Ship Date,0
Ship Mode,0
Customer ID,0
Customer Name,0
Segment,0
Country,0
City,0


In [10]:
df.duplicated().sum()

0

In [11]:
df.dtypes

,0
Row ID,int64
Order ID,object
Order Date,object
Ship Date,object
Ship Mode,object
Customer ID,object
Customer Name,object
Segment,object
Country,object
City,object


In [12]:
#Numerical Variables Analysis
df.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55190.379428,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32063.693350,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,56430.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


In [13]:
# Calculate coefficient of varation (CV) / relative standard deviation (RSD)
profit_variance_co = (234.36/28.65)*100
print(profit_variance_co)

818.0104712041885


This high CV (818%) indicates that the profit data has very high variability relative to its mean - the standard deviation is about 8.18 times the mean. This suggests:

Profit values are highly dispersed
There's significant volatility in profit numbers
The mean alone might not be a good representative measure of the data

For business context, this high CV suggests that profits are quite unstable or inconsistent across transactions, which could warrant further investigation into:

What causes some transactions to be highly profitable while others aren't
Whether certain customer segments or product categories show more consistent profits
If there are specific factors driving the high variability in profits

A more reasonable CV for profit in retail might be between 30-100%, though this varies by industry and business model. The very high CV here (818%) suggests this might be an area needing management attention.

In [ ]:

def calculate_rfm_scores(df, date_column='Order Date', customer_id='Customer ID',
                        sales_column='Sales'):
    """
    Calculate RFM (Recency, Frequency, Monetary) scores for each customer

    Parameters:
    -----------
    df : DataFrame
        Input DataFrame with customer transaction data
    date_column : str
        Name of the column containing transaction dates
    customer_id : str
        Name of the column containing customer IDs
    sales_column : str
        Name of the column containing transaction amounts

    Returns:
    --------
    DataFrame with RFM scores and segments
    """
    # Ensure date column is datetime
    df[date_column] = pd.to_datetime(df[date_column])

    # Calculate max date for recency calculation
    max_date = df[date_column].max()

    # Calculate RFM metrics
    rfm = df.groupby(customer_id).agg({
        date_column: lambda x: (max_date - x.max()).days,    # Recency
        'Order ID': 'count',                                 # Frequency
        sales_column: 'sum'                                  # Monetary
    }).reset_index()

    # Rename columns
    rfm.columns = [customer_id, 'Recency', 'Frequency', 'Monetary']

    # Create R, F, M quartile scores (1-4)
    rfm['R_Score'] = pd.qcut(rfm['Recency'], q=4, labels=[4, 3, 2, 1])
    rfm['F_Score'] = pd.qcut(rfm['Frequency'], q=4, labels=[1, 2, 3, 4])
    rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=4, labels=[1, 2, 3, 4])

    # Calculate RFM Score
    rfm['RFM_Score'] = (rfm['R_Score'].astype(str) +
                        rfm['F_Score'].astype(str) +
                        rfm['M_Score'].astype(str))

    # Segment customers based on RFM score
    def segment_customers(row):
        r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
        if r == 4 and f == 4 and m == 4:
            return 'Best Customers'
        elif (r >= 3) and (f >= 3) and (m >= 3):
            return 'Loyal Customers'
        elif (r >= 3) and (f >= 2) and (m >= 2):
            return 'Active Customers'
        elif (r >= 2) and (f >= 2) and (m >= 2):
            return 'Regular Customers'
        elif (r >= 2) and (f >= 1) and (m >= 1):
            return 'New Customers'
        else:
            return 'Lost Customers'

    rfm['Customer_Segment'] = rfm.apply(segment_customers, axis=1)

    return rfm

def validate_segments(rfm_df, features=['Recency', 'Frequency', 'Monetary']):
    """
    Perform comprehensive validation of the RFM segmentation

    Parameters:
    -----------
    rfm_df : DataFrame
        RFM DataFrame with segmentation results
    features : list
        List of feature columns used for validation

    Returns:
    --------
    Dictionary containing validation metrics and visualizations
    """
    # Standardize features for validation
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(rfm_df[features])

    # Calculate segment statistics
    segment_stats = rfm_df.groupby('Customer_Segment')[features].agg([
        'count', 'mean', 'std', 'min', 'max'
    ]).round(2)

    # Prepare visualization data
    plt.figure(figsize=(15, 10))

    # Plot segment characteristics
    for i, feature in enumerate(features, 1):
        plt.subplot(2, 2, i)
        sns.boxplot(data=rfm_df, x='Customer_Segment', y=feature)
        plt.xticks(rotation=45)
        plt.title(f'{feature} Distribution by Segment')

    plt.tight_layout()

    # Calculate segment transition probabilities (if date information available)
    # This assumes you have multiple time periods of data

    return {
        'segment_stats': segment_stats,
        'segment_sizes': rfm_df['Customer_Segment'].value_counts(),
        'segment_proportions': rfm_df['Customer_Segment'].value_counts(normalize=True)
    }

def generate_segment_profiles(rfm_df, original_df):
    """
    Generate detailed profiles for each customer segment

    Parameters:
    -----------
    rfm_df : DataFrame
        RFM DataFrame with segmentation results
    original_df : DataFrame
        Original transaction DataFrame

    Returns:
    --------
    Dictionary containing segment profiles and recommendations
    """
    profiles = {}

    for segment in rfm_df['Customer_Segment'].unique():
        # Get customers in segment
        segment_customers = rfm_df[rfm_df['Customer_Segment'] == segment]['Customer ID']

        # Get segment transactions
        segment_txns = original_df[original_df['Customer ID'].isin(segment_customers)]

        # Calculate segment metrics
        profiles[segment] = {
            'customer_count': len(segment_customers),
            'average_order_value': segment_txns.groupby('Order ID')['Sales'].sum().mean(),
            'average_items_per_order': segment_txns.groupby('Order ID')['Quantity'].sum().mean(),
            'total_revenue': segment_txns['Sales'].sum(),
            'total_profit': segment_txns['Profit'].sum(),
            'profit_margin': (segment_txns['Profit'].sum() / segment_txns['Sales'].sum()) * 100,
            'average_discount': segment_txns['Discount'].mean() * 100,
            'top_categories': segment_txns.groupby('Category')['Sales'].sum().nlargest(3)
        }

    return profiles

def plot_segment_analysis(profiles):
    """
    Create visualization dashboard for segment analysis

    Parameters:
    -----------
    profiles : dict
        Dictionary containing segment profiles
    """
    plt.figure(figsize=(20, 12))

    # Plot 1: Customer Count by Segment
    plt.subplot(2, 2, 1)
    customer_counts = [p['customer_count'] for p in profiles.values()]
    plt.bar(profiles.keys(), customer_counts)
    plt.title('Customer Count by Segment')
    plt.xticks(rotation=45)

    # Plot 2: Average Order Value
    plt.subplot(2, 2, 2)
    aov = [p['average_order_value'] for p in profiles.values()]
    plt.bar(profiles.keys(), aov)
    plt.title('Average Order Value by Segment')
    plt.xticks(rotation=45)

    # Plot 3: Profit Margin
    plt.subplot(2, 2, 3)
    margins = [p['profit_margin'] for p in profiles.values()]
    plt.bar(profiles.keys(), margins)
    plt.title('Profit Margin by Segment')
    plt.xticks(rotation=45)

    # Plot 4: Average Discount
    plt.subplot(2, 2, 4)
    discounts = [p['average_discount'] for p in profiles.values()]
    plt.bar(profiles.keys(), discounts)
    plt.title('Average Discount by Segment')
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()